# Dialogues Multi-Agents Argumentatifs

**Navigation**: [← Tweety-7b-Ranking-Probabilistic](Tweety-7b-Ranking-Probabilistic.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-9-Preferences →](Tweety-9-Preferences.ipynb)

---

## Objectifs pédagogiques

1. Comprendre le framework d'agents de TweetyProject
2. Découvrir les protocoles de dialogue argumentatif
3. Simuler des débats entre agents avec échanges structurés
4. Explorer les stratégies argumentatives et la résolution de conflits

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

## Durée estimée : 35 minutes

## Modules TweetyProject couverts

- `org.tweetyproject.agents` - Framework agents de base
- `org.tweetyproject.agents.dialogues` - Simulation de dialogues argumentatifs

In [1]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
    "SAT_SOLVER_PYTHON": "",
    "MARCO": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str:
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover)
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# 4. SAT Solver Python (CaDiCaL, Glucose via pySAT)
for sat in [pathlib.Path("../ext_tools/sat_solver.py"), pathlib.Path("ext_tools/sat_solver.py")]:
    if sat.exists():
        EXTERNAL_TOOLS["SAT_SOLVER_PYTHON"] = str(sat.resolve())
        break

# 5. MARCO (MUS enumerator avec Z3)
for mp in [pathlib.Path("../ext_tools/marco.py"), pathlib.Path("ext_tools/marco.py")]:
    if mp.exists():
        EXTERNAL_TOOLS["MARCO"] = str(mp.resolve())
        break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break

    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")

        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

--- Verification JVM Tweety + Outils ---
JDK portable: zulu17.50.19-ca-jdk17.0.11-win_x64
JVM demarree avec 35 JARs.

--- Outils disponibles ---
  CLINGO: clingo
  SPASS: SPASS.exe
  EPROVER: eprover.exe
  SAT_SOLVER_PYTHON: sat_solver.py
  MARCO: marco.py

JVM prete. Outils: 5/5


#### Interprétation de la configuration

Les résultats ci-dessus confirment que l'environnement est correctement configuré pour simuler des dialogues multi-agents:

**Configuration JVM validée:**
- **JDK portable**: Zulu 17 détecté et configuré automatiquement
- **JARs Tweety**: 35 bibliothèques chargées (incluant `agents.jar`, `arg-dung.jar`, `arg-prob.jar`)
- **Outils externes**: 5/5 disponibles (Clingo, SPASS, EProver, SAT solvers, MARCO)

**Pourquoi ces outils sont importants pour les dialogues:**
| Outil | Usage dans les dialogues |
|-------|--------------------------|
| **Clingo** | Calcul des extensions stables (ASP) pour la sémantique d'argumentation |
| **SPASS** | Raisonnement modal pour modéliser les croyances des agents |
| **EProver** | Vérification formelle des stratégies argumentatives |
| **SAT solvers** | Détermination rapide de l'acceptabilité des arguments |
| **MARCO** | Énumération des Minimal Unsatisfiable Subsets (conflits argumentatifs) |

> **Note technique**: La présence de tous les outils permet aux agents d'utiliser différents raisonneurs selon le contexte du dialogue (logique classique, modale, ou ASP).

## Partie 6 : Agents et Dialogues Argumentatifs

Cette section explore le framework multi-agents de TweetyProject et les protocoles de dialogue argumentatif. Les dialogues permettent de modéliser des échanges structurés entre agents qui défendent des positions contradictoires.

### 6.1 Framework Agents de Base

Le module `org.tweetyproject.agents` fournit les abstractions fondamentales pour la modélisation multi-agents:

- **Agent**: Classe de base représentant un agent autonome
- **Environment**: Environnement dans lequel évoluent les agents
- **Perceivable**: Interface pour les objets perceptibles
- **MultiAgentSystem**: Système regroupant plusieurs agents

Ces composants permettent de construire des simulations où des agents interagissent selon des protocoles définis.

#### Exploration des classes agents disponibles

Nous allons maintenant explorer les classes concrètes du module `agents.dialogues` pour identifier les composants utilisables dans une simulation. Cette exploration nous permettra de comprendre l'architecture disponible avant de construire un dialogue.

### 6.2 Dialogues Argumentatifs

Le module `org.tweetyproject.agents.dialogues` permet de simuler des dialogues structures ou des agents echangent des arguments. Il contient plusieurs sous-packages :

**agents.dialogues** (base) :
- **DialogueTrace** : Historique des echanges dans un dialogue
- **ExecutableExtension** : Extension executable dans un dialogue

**agents.dialogues.oppmodels** (modeles d'opposition) :
- **ArguingAgent** : Agent argumentatif principal
- **GroundedGameProtocol** : Protocole base sur la semantique grounded
- **GroundedGameSystem** : Systeme de jeu complet
- **T1/T2/T3BeliefState** : Etats de croyance a differents niveaux de recursion

**agents.dialogues.lotteries** (jeux de loterie) :
- **AbstractLotteryAgent** : Agent participant a un jeu de loterie
- **LotteryGameSystem** : Systeme gerant le jeu

Les dialogues argumentatifs suivent generalement un protocole definissant :
1. **Les coups autorises** (moves) : assertion, challenge, retract
2. **Les regles de tour** : qui peut parler et quand
3. **Les conditions de terminaison** : consensus, impasse, timeout

In [2]:
# --- 6.2 Dialogues Argumentatifs ---
print("\n--- 6.2 Dialogues Argumentatifs ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Exploration des dialogues argumentatifs...")
    dialogues_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        
        # Imports dialogues
        try:
            from org.tweetyproject.agents.dialogues import ExecutableExtension
            print("Import ExecutableExtension reussi.")
            dialogues_imports_ok = True
        except ImportError as e:
            print(f"Import direct ExecutableExtension echoue: {e}")
        
        # Exploration du package dialogues et ses sous-packages
        print("\n--- Classes disponibles dans agents.dialogues ---")
        
        # Structure des packages dialogues (noms corrects Tweety 1.29)
        known_classes = {
            "agents.dialogues": [
                "ExecutableExtension",
                "DialogueTrace",
            ],
            "agents.dialogues.lotteries": [
                "AbstractLotteryAgent",
                "LotteryGameSystem",
            ],
            "agents.dialogues.oppmodels": [
                "ArguingAgent",           # Agent argumentatif (pas ArgumentativeAgent)
                "GroundedGameProtocol",   # Protocole grounded (pas GroundedSolverProtocol)
                "GroundedGameSystem",     # Systeme de jeu complet
                "T1BeliefState",          # Etats de croyance niveaux 1-3
                "T2BeliefState",
                "T3BeliefState",
            ],
        }

        print("Classes par package:")
        for package, classes in known_classes.items():
            print(f"\n  {package}:")
            for cls_name in classes:
                try:
                    full_name = f"org.tweetyproject.{package}.{cls_name}"
                    cls = jpype.JClass(full_name)
                    print(f"    [OK] {cls_name}")
                except Exception:
                    print(f"    [--] {cls_name}")
                    
        dialogues_imports_ok = True

        # --- Exemple de Dialogue Argumentatif ---
        if dialogues_imports_ok:
            print("\n--- Modele Conceptuel de Dialogue (oppmodels) ---")
            print("""
Dans un dialogue argumentatif typique (package oppmodels):

1. Deux agents (ArguingAgent) debattent sur un Dung Framework
2. Chaque agent a un etat de croyance (T1/T2/T3 BeliefState)
3. Le protocole (GroundedGameProtocol) gere les tours
4. Le systeme (GroundedGameSystem) coordonne le jeu

Niveaux d'etat de croyance:
- T1: L'agent connait le framework complet
- T2: L'agent a des croyances sur les croyances adverses
- T3: Recursion a 3 niveaux (croyances sur croyances sur croyances)

Coups autorises dans le jeu grounded:
- ASSERT(argument) : Affirmer un argument
- CHALLENGE(argument) : Contester un argument  
- RETRACT : Se retirer du debat

Le vainqueur est determine par la semantique grounded.
""")

    except Exception as e_gen:
        print(f"Erreur Python inattendue: {e_gen}")
        import traceback; traceback.print_exc()


--- 6.2 Dialogues Argumentatifs ---
JVM prete. Exploration des dialogues argumentatifs...
Import ExecutableExtension reussi.

--- Classes disponibles dans agents.dialogues ---
Classes par package:

  agents.dialogues:
    [OK] ExecutableExtension
    [OK] DialogueTrace

  agents.dialogues.lotteries:
    [OK] AbstractLotteryAgent
    [OK] LotteryGameSystem

  agents.dialogues.oppmodels:
    [OK] ArguingAgent
    [OK] GroundedGameProtocol
    [OK] GroundedGameSystem
    [OK] T1BeliefState
    [OK] T2BeliefState
    [OK] T3BeliefState

--- Modele Conceptuel de Dialogue (oppmodels) ---

Dans un dialogue argumentatif typique (package oppmodels):

1. Deux agents (ArguingAgent) debattent sur un Dung Framework
2. Chaque agent a un etat de croyance (T1/T2/T3 BeliefState)
3. Le protocole (GroundedGameProtocol) gere les tours
4. Le systeme (GroundedGameSystem) coordonne le jeu

Niveaux d'etat de croyance:
- T1: L'agent connait le framework complet
- T2: L'agent a des croyances sur les croyances 

#### Construction d'un scénario de loterie

Maintenant que nous avons compris l'architecture des dialogues, explorons un cas d'usage concret: les **jeux de loterie argumentative**. Dans ce contexte, les agents doivent prendre des décisions sous incertitude en évaluant les probabilités d'acceptation de leurs arguments.

Nous allons créer un scénario réaliste avec trois propositions en conflit et calculer les probabilités d'acceptation pour guider la stratégie des agents.

#### Analyse des classes disponibles

L'exploration ci-dessus confirme que TweetyProject 1.29 fournit une architecture complete pour les dialogues argumentatifs :

**Package agents.dialogues (base) :**
- `ExecutableExtension` : Extension qui peut etre executee comme un coup dans un dialogue
- `DialogueTrace` : Historique complet des echanges (moves, turns, etats)

**Package agents.dialogues.oppmodels (jeux grounded) :**
- `ArguingAgent` : Agent capable de jouer au jeu grounded (prouver qu'un argument est dans l'extension grounded)
- `GroundedGameProtocol` : Regles du jeu - definit les coups legaux et les transitions
- `GroundedGameSystem` : Orchestre le jeu entre deux agents
- `T1/T2/T3BeliefState` : Modeles de croyance a recursion croissante (Theory of Mind)

**Package agents.dialogues.lotteries :**
- `AbstractLotteryAgent` : Agent prenant des decisions sous incertitude
- `LotteryGameSystem` : Systeme gerant les jeux de loterie argumentative

**Niveaux de croyance :**
- **T1** : L'agent connait le framework complet (omniscient)
- **T2** : L'agent a des croyances sur ce que l'adversaire croit
- **T3** : Recursion a 3 niveaux - permet des strategies sophistiquees

Ces classes permettent de simuler des debats ou chaque agent essaie de prouver l'acceptabilite de son argument cible.

### 6.3 Jeux de Loterie Argumentative

Les jeux de loterie argumentative (`agents.dialogues.lotteries`) combinent l'argumentation probabiliste avec les dialogues multi-agents:

- **AbstractLotteryAgent**: Agent participant à un jeu de loterie
- **LotteryGameSystem**: Système gérant le jeu
- **UtilityFunction**: Fonction d'utilité pour évaluer les résultats

Ces jeux permettent de modéliser des situations de négociation où les agents ont des préférences sur les issues possibles du dialogue.

In [3]:
# --- 6.3 Jeux de Loterie Argumentative ---
print("\n--- 6.3 Jeux de Loterie Argumentative ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Exploration des jeux de loterie...")
    try:
        import jpype
        from jpype.types import *
        
        # Imports nécessaires
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.semantics import Semantics
        from org.tweetyproject.arg.prob.lotteries import SubgraphProbabilityFunction, ArgumentationLottery, UtilityFunction
        from org.tweetyproject.arg.dung.divisions import Division
        from org.tweetyproject.arg.dung.semantics import Extension
        
        print("Imports pour loteries reussis.")
        
        # Créer un framework simple
        print("\n--- Creation d'un scenario de loterie ---")
        theory = DungTheory()
        a = Argument("proposition_A"); b = Argument("proposition_B"); c = Argument("evidence_C")
        theory.add(a); theory.add(b); theory.add(c)
        theory.add(Attack(a, b)); theory.add(Attack(b, a)); theory.add(Attack(c, b))
        
        print(f"Framework: {theory}")
        print("""
Scenario:
- Agent 1 supporte proposition_A
- Agent 2 supporte proposition_B  
- evidence_C attaque B, favorisant A
""")
        
        # Distribution de probabilité
        prob_func = SubgraphProbabilityFunction(theory)
        print(f"\nNombre de configurations possibles: {prob_func.size()}")
        
        # Calcul des probabilités d'acceptation
        target_sem = Semantics.GROUNDED_SEMANTICS
        try:
            p_a = prob_func.getAcceptanceProbability(a, target_sem).doubleValue()
            p_b = prob_func.getAcceptanceProbability(b, target_sem).doubleValue()
            p_c = prob_func.getAcceptanceProbability(c, target_sem).doubleValue()
            
            print(f"\nProbabilites d'acceptation (Grounded):")
            print(f"  P(proposition_A accepte) = {p_a:.3f}")
            print(f"  P(proposition_B accepte) = {p_b:.3f}")
            print(f"  P(evidence_C accepte)    = {p_c:.3f}")
            
            print(f"""
Interpretation:
- Si evidence_C est presente, B est attaque et A gagne
- La probabilite que A soit accepte ({p_a:.1%}) > celle de B ({p_b:.1%})
- Un agent rationnel devrait investir pour maintenir C dans le debat
""")
        except Exception as e_prob:
            print(f"Erreur calcul probabilites: {e_prob}")
        
    except ImportError as e:
        print(f"Erreur d'import pour loteries: {e}")
    except jpype.JException as e_java:
        print(f"Erreur Java: {e_java.message()}")
    except Exception as e_gen:
        print(f"Erreur Python inattendue: {e_gen}")
        import traceback; traceback.print_exc()


--- 6.3 Jeux de Loterie Argumentative ---
JVM prete. Exploration des jeux de loterie...
Imports pour loteries reussis.

--- Creation d'un scenario de loterie ---
Framework: <{ proposition_A, proposition_B, evidence_C },[(proposition_B,proposition_A), (proposition_A,proposition_B), (evidence_C,proposition_B)]>

Scenario:
- Agent 1 supporte proposition_A
- Agent 2 supporte proposition_B  
- evidence_C attaque B, favorisant A


Nombre de configurations possibles: 19

Probabilites d'acceptation (Grounded):
  P(proposition_A accepte) = 0.526
  P(proposition_B accepte) = 0.316
  P(evidence_C accepte)    = 0.632

Interpretation:
- Si evidence_C est presente, B est attaque et A gagne
- La probabilite que A soit accepte (52.6%) > celle de B (31.6%)
- Un agent rationnel devrait investir pour maintenir C dans le debat



#### Analyse du scenario de loterie

Les resultats ci-dessus illustrent comment les probabilites guident la strategie d'un agent rationnel :

**Configuration du debat :**
- `proposition_A` et `proposition_B` s'attaquent mutuellement (conflit direct)
- `evidence_C` attaque `proposition_B`, brisant la symetrie en faveur de A

**Interpretation des probabilites :**
| Argument | Probabilite | Raison |
|----------|-------------|--------|
| `evidence_C` | 63.2% | Jamais attaque, donc souvent accepte |
| `proposition_A` | 52.6% | Defendu par C quand C est present |
| `proposition_B` | 31.6% | Attaque par A et C, rarement accepte |

**Strategie optimale :**
Un agent rationnel supportant A devrait :
1. **Proteger** `evidence_C` car sa presence garantit presque l'acceptation de A
2. **Anticiper** les tentatives adverses d'attaquer C
3. **Investir** ses ressources dans le maintien de C plutot que dans la defense directe de A

**Lien avec la theorie des jeux :**
Cette analyse probabiliste permet de calculer l'**utilite attendue** de chaque strategie et de choisir la meilleure action selon le critere de decision (ex: maximiser l'esperance).

---

## Résumé

Ce notebook a introduit le framework multi-agents et les dialogues argumentatifs:

| Composant | Description | Classes TweetyProject |
|-----------|-------------|----------------------|
| **Agents de base** | Abstraction d'agents autonomes | `Agent`, `DummyAgent`, `MultiAgentSystem` |
| **Dialogues** | Protocoles d'échange structurés | `DialogueTrace`, `ExecutableExtension` |
| **Loteries** | Argumentation probabiliste + décision | `AbstractLotteryAgent`, `LotteryGameSystem` |

### Architecture d'un dialogue argumentatif

```
Agent A                    Agent B
   |                          |
   |------ ASSERT(arg1) ----->|
   |                          |
   |<----- CHALLENGE(arg1) ---|
   |                          |
   |------ ASSERT(arg2) ----->|  (défense de arg1)
   |                          |
   |<----- ACCEPT / ATTACK ---|
   |                          |
         ... (tours suivants)
```

### Points clés

1. **MultiAgentSystem** gère l'interaction entre plusieurs agents
2. Les **protocoles** définissent les règles du dialogue (moves autorisés, tours, terminaison)
3. Les **loteries** permettent la prise de décision rationnelle quand l'issue est incertaine
4. L'**acceptabilité probabiliste** guide les stratégies des agents

### Limitations

- Le module `agents.dialogues` est conçu pour être étendu avec des classes concrètes
- L'implémentation complète nécessite de définir des protocoles spécifiques
- Les exemples Java de TweetyProject fournissent des implémentations de référence

## Prochaines étapes

Le notebook suivant explore le module **Preferences** pour l'agrégation de préférences et la théorie du vote.

---

**Navigation**: [← Tweety-7b-Ranking-Probabilistic](Tweety-7b-Ranking-Probabilistic.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-9-Preferences →](Tweety-9-Preferences.ipynb)